In [2]:
import requests as r
from bs4 import BeautifulSoup
import re
import string
import csv

# Get Champion Voiceline Data

In [3]:
# Let's get the latest patch and get the data from tehre
patch = r.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
ddragon = r.get(f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/champion.json").json()
champions = [ddragon['data'][champ]['name'] for champ in ddragon['data'].keys()]
print(champions)

['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'Aurelion Sol', 'Azir', 'Bard', "Bel'Veth", 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia', "Cho'Gath", 'Corki', 'Darius', 'Diana', 'Draven', 'Dr. Mundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'Jarvan IV', 'Jax', 'Jayce', 'Jhin', 'Jinx', "Kai'Sa", 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', "Kha'Zix", 'Kindred', 'Kled', "Kog'Maw", "K'Sante", 'LeBlanc', 'Lee Sin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'Master Yi', 'Milio', 'Miss Fortune', 'Wukong', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu & Willump', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiya

In [4]:
for champion in champions:
    html = r.get(f"https://leagueoflegends.fandom.com/wiki/{champion}/LoL/Audio")
    if html.status_code != 200:
        print("❌ " + champion + " " + str(html.status_code))
    else:
        print("✅ " + champion)

Aatrox ✅
Ahri ✅
Akali ✅
Akshan ✅
Alistar ✅
Amumu ✅
Anivia ✅
Annie ✅
Aphelios ✅
Ashe ✅
Aurelion Sol ✅
Azir ✅
Bard ✅
Bel'Veth ✅
Blitzcrank ✅
Brand ✅
Braum ✅
Caitlyn ✅
Camille ✅
Cassiopeia ✅
Cho'Gath ✅
Corki ✅
Darius ✅
Diana ✅
Draven ✅
Dr. Mundo ✅
Ekko ✅
Elise ✅
Evelynn ✅
Ezreal ✅
Fiddlesticks ✅
Fiora ✅
Fizz ✅
Galio ✅
Gangplank ✅
Garen ✅
Gnar ✅
Gragas ✅
Graves ✅
Gwen ✅
Hecarim ✅
Heimerdinger ✅
Illaoi ✅
Irelia ✅
Ivern ✅
Janna ✅
Jarvan IV ✅
Jax ✅
Jayce ✅
Jhin ✅
Jinx ✅
Kai'Sa ✅
Kalista ✅
Karma ✅
Karthus ✅
Kassadin ✅
Katarina ✅
Kayle ✅
Kayn ✅
Kennen ✅
Kha'Zix ✅
Kindred ✅
Kled ✅
Kog'Maw ✅
K'Sante ✅
LeBlanc ✅
Lee Sin ✅
Leona ✅
Lillia ✅
Lissandra ✅
Lucian ✅
Lulu ✅
Lux ✅
Malphite ✅
Malzahar ✅
Maokai ✅
Master Yi ✅
Milio ✅
Miss Fortune ✅
Wukong ✅
Mordekaiser ✅
Morgana ✅
Nami ✅
Nasus ✅
Nautilus ✅
Neeko ✅
Nidalee ✅
Nilah ✅
Nocturne ✅
Nunu & Willump 404
Olaf ✅
Orianna ✅
Ornn ✅
Pantheon ✅
Poppy ✅
Pyke ✅
Qiyana ✅
Quinn ✅
Rakan ✅
Rammus ✅
Rek'Sai ✅
Rell ✅
Renata Glasc ✅
Renekton ✅
Rengar ✅
Riven ✅
Rumble

In [4]:
corrections = {
    "Nunu & Willump": "Nunu"
}

In [5]:
lol_champions = list(map(corrections.get, champions, champions))

### Scrape the LoL Wiki and filter out the empty/SFX items

In [60]:
clean_actions = ["Effort sound.", "Death sound.", "Effort sound", "Death sound"]
def filter_voicelines(voiceline):
    # Remove empty voicelines
    if voiceline is None or voiceline == "":
        return False
    # Remove above actions since they are sound effects
    if voiceline in clean_actions:
        return False
    # Remove stated sound effects
    if voiceline.split(" ")[-1] == "SFX":
        return False
    # Remove non-transcribed (Ivern sadly has a particularly high number of these)
    if len(voiceline) > 4 and voiceline[-4:] == ".ogg":
        return False
    return True

In [11]:
# League of Legends wiki voiceline format
# <ul>
#     <li>
#         <i>VOICELINE</i>
#    </li>
# </ul>

raw_voicelines = {}
for champion in lol_champions:
    print(f"Extracting voicelines from {champion}")
    html = r.get(f"https://leagueoflegends.fandom.com/wiki/{champion}/LoL/Audio")
    # First we must remove the trivia section since it does not contain new voice lines and messes with the scraping really hard
    no_trivia = re.sub(r'<span class="mw-headline" id="Trivia">.*', '', html.text, flags=re.DOTALL).strip()
    soup = BeautifulSoup(no_trivia, 'html.parser')
    unsorted_lines = soup.find('div', class_="mw-parser-output").find_all('ul')
    voice_lines = []
    for ul in unsorted_lines:
        list_elements = ul.find_all('li')
        for ol in list_elements:
            voice_lines.extend(map(lambda x:x.text, ol.find_all('i')))
# Remove duplicates and empty lines
    voice_lines = set(list(filter(filter_voicelines, voice_lines)))
    raw_voicelines[champion] = voice_lines

Extracting voicelines from Aatrox
Extracting voicelines from Ahri
Extracting voicelines from Akali
Extracting voicelines from Akshan
Extracting voicelines from Alistar
Extracting voicelines from Amumu
Extracting voicelines from Anivia
Extracting voicelines from Annie
Extracting voicelines from Aphelios
Extracting voicelines from Ashe
Extracting voicelines from Aurelion Sol
Extracting voicelines from Azir
Extracting voicelines from Bard
Extracting voicelines from Bel'Veth
Extracting voicelines from Blitzcrank
Extracting voicelines from Brand
Extracting voicelines from Braum
Extracting voicelines from Caitlyn
Extracting voicelines from Camille
Extracting voicelines from Cassiopeia
Extracting voicelines from Cho'Gath
Extracting voicelines from Corki
Extracting voicelines from Darius
Extracting voicelines from Diana
Extracting voicelines from Draven
Extracting voicelines from Dr. Mundo
Extracting voicelines from Ekko
Extracting voicelines from Elise
Extracting voicelines from Evelynn
Extra

In [87]:
# We need to manually extract Kayle/Morgana's interactions and Rakan/Xayah's interactions.
# Kayle/Morg were done manually while Rakan/Xayah were done using this script. This honestly can be used in
# future as the main pulling script since it's alot more efficient.
html = r.get(f"https://leagueoflegends.fandom.com/wiki/Xayah/LoL/Audio")
no_trivia = re.sub(r'<span class="mw-headline" id="Trivia">.*', '', html.text, flags=re.DOTALL).strip()
soup = BeautifulSoup(no_trivia, 'html.parser')
unsorted_lines = soup.find('div', class_="mw-parser-output")
rakan = []
xayah = []
for _ in unsorted_lines.find_all('li'):
    champ = _.b
    if champ and not champ.find_parent('i'):
        line = _.i
        if champ.text in ["Xayah and Rakan:", "Xayah:"]:
#             print(champ.text, line.text)
            xayah.append(line.text)
        if champ.text in ["Xayah and Rakan:", "Rakan:"]:
#             print(champ.text, line.text)
            rakan.append(line.text)
    else:
        if _.i:
#             print("Adding this line", _.i.text)
            xayah.append(_.i.text)

In [89]:
voice_lines = set(list(filter(filter_voicelines, xayah)))

In [90]:
with open("xayahflag.csv", "w", newline='', encoding='utf-8-sig') as flag:
    with open("xayah.csv", "w", newline='', encoding='utf-8-sig') as file:
        csv_writer = csv.writer(file, dialect='excel')
        flag_writer = csv.writer(flag, dialect='excel')
        for _ in voice_lines:
            categorized = categorize_voicelines("Xayah", _)
            if categorized[2] == "FLAG":
                flag_writer.writerow(categorized)
            else:
                csv_writer.writerow(categorized)

#### Cleanup Voice Lines

In [91]:
sum(len(_) for _ in raw_voicelines.values())

NameError: name 'raw_voicelines' is not defined

In [7]:
def standardize_punctuation(_):
    return _.replace('\u200b', '').replace('...', '…').encode(encoding = 'UTF-8', errors='ignore').decode("utf-8")
def remove_punct(_):
    return _.translate(str.maketrans('','',string.punctuation))
def categorize_voicelines(champion, voiceline):
    if voiceline[0] == '"' and voiceline[-1] == '"':
        voiceline = voiceline[1:-1]
        line_type = "voiceline"
    elif voiceline in clean_actions:
        line_type = "action"
    elif remove_punct(voiceline.split(" ")[0]) == remove_punct(champion):
        line_type = "action"
    elif voiceline[0] in ["(", "*"] or voiceline[-1] in [")", "*"]:
        pattern = "\*([^*]+)\*|\(([^)]+)\)"
        voiceline_no_action = re.sub(pattern, '', voiceline).strip()
        if len(voiceline_no_action) > 0 and voiceline_no_action[0] == '"' and voiceline_no_action[-1] == '"':
            line_type = "voiceline"
        else:
            line_type = "FLAG"
    else:
        line_type = "FLAG"
    return [champion, standardize_punctuation(voiceline), line_type]

In [131]:
with open("flagged.csv", "w", newline='\n', encoding='utf-8') as flag:
    with open("voicelines.csv", "w", newline='\n', encoding='utf-8') as file:
        csv_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        flag_writer = csv.writer(flag, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for champion, voicelines in raw_voicelines.items():
            for _ in voicelines:
                categorized = categorize_voicelines(champion, _)
                if categorized[2] == "FLAG":
                    flag_writer.writerow(categorized)
                else:
                    csv_writer.writerow(categorized)

In [142]:
# Once we're done manually sorting through the flagged items
df1 = pd.read_csv("flagged.csv", encoding= 'unicode_escape', header=None)
df2 = pd.read_csv("voicelines.csv", encoding= 'unicode_escape', header=None)
full_df = pd.concat([df1,df2])
unique_df = full_df.drop_duplicates(keep='last')

In [149]:
unique_df.sort_values(2).to_csv('test.csv')

In [ ]:
# From this point on, the editing was done manually with Excel.

# Notes:
# Non-character's that have voicelines; kinda...
# PEARL, Kiko, Pythagoras, Shiro, Kuro, Ina, Riggle, Fuemigo, "MF/EXO"
# I have included both MF/EXO voice lines under Miss Fortune.

# Manually entered: 
# Kayle/Morgana interactions
# Rhaast/Shadow Assasin actions
# Lamb/Wolf/Kindred

# Many things are standardized for 90% of the wiki... such as using … in most places instead of ...
# or having periods at the end of actions. However, there are many places where the standard is not kept.
# The most egregious example of formatting issues is on the Evelynn voiceline page which contains
# many zero width spaces and formatting errors. Perhaps I will work with the wiki team to fix these in the future.